# Globus Auth for Applications and Humans

In this notebook, we'll show the difference between an application that can
do things as itself and an application that can login humans and do things
with their identity and permissions.

## Requirements
- Install the [Globus Python SDK](https://globus-sdk-python.readthedocs.io/en/stable/installation.html).
- Join the [Tutorial Users Group](https://app.globus.org/groups/50b6a29c-63ac-11e4-8062-22000ab68755) in order to experiment with sharing/guest collections.

## Globus SDK References
- Source code: https://github.com/globus/globus-sdk-python
- Documentation: https://globus-sdk-python.readthedocs.io/en/stable/

In [ ]:
import globus_sdk
from globus_sdk import AuthClient, AccessTokenAuthorizer
import json  # just so we can pretty-print response data

# Let's first do some things as the application itself.

Applications can use Globus APIs with their own application identities. This is
useful for automation, and also for things data repositories have to do without
a human being driving the process.

* We have to identify the application with an application identity.
* We have to request tokens for the APIs we plan to use.
* Authentication is 100% non-interactive!
* We have to organize the tokens we get back

Once we have tokens, we can then use Globus APIs to do things, but everything is
done using the application's identity, so only permisions that have been granted
to the application are available.

In [ ]:
# You'll need to register an application with Globus to get the Client ID and Secret below.
# Go to https://app.globus.org/settings/developers and select "Register a service account or 
# application credential for automation." Then copy-paste the Client ID and Client Secret here.

APP_CLIENT_ID = "..."
# the secret, loaded from wherever you store it
APP_CLIENT_SECRET = "..."

# Create a client object for things my app does as itself (application identity)
myappclient = globus_sdk.ConfidentialAppAuthClient(
    client_id=APP_CLIENT_ID, client_secret=APP_CLIENT_SECRET
)

In [ ]:
# We can authenticate the app itself and get its tokens non-interactively

token_response = myappclient.oauth2_client_credentials_tokens()

# the useful values that you want at the end of this
globus_auth_data = token_response.by_resource_server["auth.globus.org"]
globus_transfer_data = token_response.by_resource_server["transfer.api.globus.org"]
print("Here's how the Auth API token is delivered to us:")
print(json.dumps(globus_auth_data, indent=2))
print("\nHere's how the Transfer API token is delivered to us:")
print(json.dumps(globus_transfer_data, indent=2))

In [ ]:
# Now that we've authenticated the application, we can
# create a transfer client to let the app use the Transfer API as itself
scopes = "urn:globus:auth:scope:transfer.api.globus.org:all"
cc_authorizer = globus_sdk.ClientCredentialsAuthorizer(myappclient, scopes)
# create a new client
mytransferclient = globus_sdk.TransferClient(authorizer=cc_authorizer)

In [ ]:
# Ok, let's see which collections have been shared with the app!

print("Endpoints shared with {}@clients.auth.globus.org:".format(APP_CLIENT_ID))
for ep in mytransferclient.endpoint_search(filter_scope="shared-with-me"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))

# Let's log someone in!

Here, we're going to use Globus Auth to authenticate a human being.
* We need to identity the application with an application ID.
* We need to tell Globus which APIs we want to be able to use the human's identity with.
* We need to gather up the access tokens Globus returns so we can use them with APIs.

Once we've done all of the above, we will then use the tokens to do a few things as the human.

In [ ]:
# Now, we're going to log in humans and do things as them. 
# Because this is a notebook, the LOGIN_CLIENT_ID doesn't get to have a secret.
LOGIN_CLIENT_ID = "..."

# Create a client object for user login flows (to allow us do things as the human user)
myloginclient = globus_sdk.NativeAppAuthClient(LOGIN_CLIENT_ID)

In [ ]:
# The first thing we have to do is gather up all the API scopes we want the user to
# grant my application. 

# Auth scopes for obtaining human identity info
openid_scope = globus_sdk.scopes.data.AuthScopes.openid
email_scope = globus_sdk.scopes.data.AuthScopes.email
profile_scope = globus_sdk.scopes.data.AuthScopes.profile

# GCS Data Access scopes for tutorial collections.
tutorial_collection_1 = "6c54cade-bde5-45c1-bdea-f4bd71dba2cc"  # collection "Globus Tutorial Collection 1"
tutorial_collection_2 = "31ce9ba0-176d-45a5-add3-f37d233ba47d"  # collection "Globus Tutorial Collection 2"
# This is the Tutorial GCS endpoint, which will be used for creating new Guest Collections
tutorial_endpoint_1 = "261692a4-7e49-4def-b59e-cbfc65e3907d"
tc1_data_access = globus_sdk.scopes.GCSCollectionScopeBuilder(tutorial_collection_1).data_access
tc2_data_access = globus_sdk.scopes.GCSCollectionScopeBuilder(tutorial_collection_2).data_access

# Add scopes as dependencies for the transfer scope, allowing transfer operations to access data
# on both tutorial mapped collections.
transfer_scope = globus_sdk.scopes.TransferScopes.make_mutable("all")
transfer_scope.add_dependency(tc1_data_access)
transfer_scope.add_dependency(tc2_data_access)

# This scope will be used to manage the Endpoint on Tutorial 1, allowing us to Create/Update/Delete
# Guest Collections on this endpoint.
tutorial_ep1_manage_collections = globus_sdk.scopes.GCSEndpointScopeBuilder(tutorial_endpoint_1).make_mutable("manage_collections")
tutorial_ep1_manage_collections.add_dependency(tc1_data_access)

requested_scopes = [openid_scope, email_scope, profile_scope, transfer_scope, tutorial_ep1_manage_collections]

In [ ]:
# Now that we know what we're asking permission to do, log the human in!

myloginclient.oauth2_start_flow(requested_scopes=requested_scopes, refresh_tokens=False)
print(f"Login Here:\n\n{myloginclient.oauth2_get_authorize_url()}")
print("\nIMPORTANT NOTE: the link above can only be used once!")
print("If login or a later step in the flow fails, you must execute this cell again to generate a new link.")
auth_code = input("PASTE YOUR CODE HERE> ")
tokens = myloginclient.oauth2_exchange_code_for_tokens(auth_code).by_resource_server
print("Tokens Received!")

In [ ]:
# Let's take a look at the tokens we received for this human
human_auth_data = tokens["auth.globus.org"]
human_transfer_data = tokens["transfer.api.globus.org"]

print("Here's how the Auth API token is delivered to us:")
print(json.dumps(human_auth_data, indent=2))
print("\nHere's how the Transfer API token is delivered to us:")
print(json.dumps(human_transfer_data, indent=2))

In [ ]:
# Extract the access token for the Globus Transfer service, known as "transfer.api.globus.org"
human_transfer_access_token = human_transfer_data['access_token']
human_transfer_authorizer = globus_sdk.AccessTokenAuthorizer(human_transfer_access_token)
human_transfer_client = globus_sdk.TransferClient(authorizer=human_transfer_authorizer)

human_auth_access_token = human_auth_data['access_token']
human_auth_authorizer = globus_sdk.AccessTokenAuthorizer(human_auth_access_token)
human_auth_client = globus_sdk.AuthClient(authorizer=human_auth_authorizer)

In [ ]:
# Let's find out who this human is.

human_userinfo = human_auth_client.oauth2_userinfo()
print(json.dumps(human_userinfo.data, indent=2))

In [ ]:
# Ok, let's see which collections have been shared with the human!

print("Endpoints shared with {}:".format(human_userinfo.data["preferred_username"]))
for ep in human_transfer_client.endpoint_search(filter_scope="shared-with-me"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))